In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
df_daily_activity = pd.read_csv("/kaggle/input/bellabeat-dataset/dailyActivity_merged.csv")
df_daily_calories = pd.read_csv("/kaggle/input/bellabeat-dataset/dailyCalories_merged.csv")
df_daily_steps = pd.read_csv("/kaggle/input/bellabeat-dataset/dailySteps_merged.csv")
df_sleep_day = pd.read_csv("/kaggle/input/bellabeat-dataset/sleepDay_merged.csv")

In [ ]:
### Check for null values in all the files
df_daily_activity.head(2)

In [ ]:
df_daily_activity.isnull().sum()

In [ ]:
df_daily_calories.head(2)

In [ ]:
df_daily_calories.isnull().sum()

In [ ]:
df_daily_steps.head(2)

In [ ]:
df_daily_steps.isnull().sum()

In [ ]:
df_sleep_day.head(2)

In [ ]:
df_sleep_day.isnull().sum()

In [ ]:
### Check for unique Id in all the files

In [ ]:
df_daily_activity.shape

In [ ]:
df_daily_calories.shape

In [ ]:
df_daily_activity['Id'].nunique()

In [ ]:
df_daily_calories['Id'].nunique()

In [ ]:
df_daily_steps.shape

In [ ]:
df_daily_steps['Id'].nunique()

In [ ]:
df_sleep_day.shape

In [ ]:
df_sleep_day['Id'].nunique()

In [ ]:
###Some quick summary statistics we'd want to know about each data frame

In [ ]:
df_daily_activity[['TotalSteps','TotalDistance','SedentaryMinutes']].describe().transpose()

In [ ]:
df_sleep_day.head(2)

In [ ]:
df_sleep_day[['TotalSleepRecords','TotalMinutesAsleep','TotalTimeInBed']].describe().transpose()

In [ ]:
sns.scatterplot(x = 'TotalSteps',y = 'SedentaryMinutes',data = df_daily_activity)

In [ ]:
sns.scatterplot(data = df_sleep_day,x = 'TotalMinutesAsleep', y = 'TotalTimeInBed')

In [ ]:
### Merging datasets

In [ ]:
df_daily_activity['Id'] = df_daily_activity['Id'].apply(pd.to_numeric)
df_sleep_day['Id'] = df_sleep_day['Id'].apply(pd.to_numeric)
df_sleep_day['SleepDay'] = df_sleep_day['SleepDay'].apply(pd.to_datetime)
df_daily_activity['ActivityDate'] = df_daily_activity['ActivityDate'].apply(pd.to_datetime)

In [ ]:
df_sleep_day['SleepDay'] = pd.to_datetime(df_sleep_day['SleepDay'].dt.strftime('%m/%d/%Y'))

In [ ]:
combined_data = df_daily_activity[['Id','ActivityDate','TotalSteps']].merge(df_sleep_day[['Id','SleepDay','TotalMinutesAsleep']],how = 'inner',left_on=['Id','ActivityDate'],right_on=['Id','SleepDay'])
combined_data.head(2)

In [ ]:
combined_data.shape

In [ ]:
df_group_id = combined_data.groupby('Id')

In [ ]:
df_group_id.describe().head(2)

In [ ]:
combined_data['TotalSteps'].max()

In [ ]:
combined_data['TotalMinutesAsleep'].max()

In [ ]:
sns.scatterplot(data = combined_data, x= 'TotalSteps',y = 'TotalMinutesAsleep')

In [ ]:
df_daily_activity = pd.read_csv("/kaggle/input/bellabeat-dataset/dailyActivity_merged.csv")
df_sleep_day = pd.read_csv("/kaggle/input/bellabeat-dataset/sleepDay_merged.csv")

In [ ]:
sleep_edit = df_sleep_day.copy()

In [ ]:
sleep_edit['Date'] = pd.to_datetime(sleep_edit['SleepDay']).dt.date
sleep_edit['Time'] = pd.to_datetime(sleep_edit['SleepDay']).dt.time
sleep_edit.drop('SleepDay',axis = 1, inplace = True)
sleep_edit.head(2)

In [ ]:
df_daily_activity['ActivityDate'] = df_daily_activity['ActivityDate'].apply(pd.to_datetime)
sleep_edit['Date'] = sleep_edit['Date'].apply(pd.to_datetime)
sleep_edit['Date'] = pd.to_datetime(sleep_edit['Date'].dt.strftime('%m/%d/%Y'))
combined_sleep = df_daily_activity.merge(sleep_edit, left_on=['Id','ActivityDate'],right_on=['Id','Date'],how = 'left')

In [ ]:
combined_sleep.shape

In [ ]:
combined_sleep.isnull().sum()

In [ ]:
only_sleep = df_daily_activity.merge(sleep_edit, left_on=['Id','ActivityDate'],right_on=['Id','Date'])
only_sleep.shape

In [ ]:
only_sleep.isnull().sum()

In [ ]:
only_sleep['Non_sleep_hrs'] = (only_sleep['SedentaryMinutes'] + only_sleep['LightlyActiveMinutes'] + only_sleep['FairlyActiveMinutes'] + only_sleep['VeryActiveMinutes'])/60
only_sleep['Total'] = only_sleep['Non_sleep_hrs'] + (only_sleep['TotalTimeInBed']/60)
only_sleep.head(2)

In [ ]:
only_sleep.columns

In [ ]:
only_sleep_clean = only_sleep.drop(only_sleep[only_sleep['Total'] > 24].index)
only_sleep_clean.head(2)

In [ ]:
only_sleep_clean.shape

In [ ]:
# Heaviely active vs Lightly active
# People who are fairly active and very active for equal to or more than 1 hour are heaviely active else lightly active

In [ ]:
only_sleep_clean['Heavy_mins'] = only_sleep_clean['VeryActiveMinutes'] + only_sleep_clean['FairlyActiveMinutes']
only_sleep_clean['Lite_mins'] = (only_sleep_clean['Total']*60) - only_sleep_clean['Heavy_mins']
only_sleep_clean['Type'] = np.where(only_sleep_clean['Heavy_mins'] >= 60,'Heavy user','Light User')
only_sleep_clean.head(2)

In [ ]:
only_sleep_clean.shape

In [ ]:
sns.countplot(data = only_sleep_clean,x = 'Type')

In [ ]:
# We can clearly see that most of the people are light users

In [ ]:
sns.boxplot(data = only_sleep_clean,x = 'Type',y = 'Calories')

In [ ]:
# Heavy users have burned more calories

In [ ]:
sns.lmplot(data = only_sleep_clean,x = 'TotalDistance', y = 'Calories',hue = 'Type')

In [ ]:
only_sleep_clean['SleepType'] = np.where(only_sleep_clean.TotalMinutesAsleep/60 < 6 ,'Less Sleep',(np.where(only_sleep_clean.TotalMinutesAsleep/60 > 8,'High Sleep','Moderate Sleep')))
only_sleep_clean.head()

In [ ]:
sns.countplot(data = only_sleep_clean,x = 'SleepType')

In [ ]:
sns.countplot(data = only_sleep_clean,hue = 'Type',x = 'SleepType')

In [ ]:
sns.boxplot(data = only_sleep_clean,x = 'SleepType',y = 'TotalMinutesAsleep',hue = 'Type')

* With higher total time in bed, Total sleeping minutes goes higher
* No relation between total sleeping minutes and total steps
* Most of the people are not active(Light users).
* People who are more active(Heavy users) burn more calories
* With goes total distance high, Calories burn also goes high
* More people sleep between 6-8 hours(Moderate sleep)
* Light users count is very high in Moderate sleep and high sleep, It is also high in less sleep

In [ ]:
combined_data.to_csv("C:\\Users\\user\\Documents\\Data Analytics\\Case Study\\Case Study 2\\bellabeat.csv", index = False)
only_sleep_clean.to_csv("C:\\Users\\user\\Documents\\Data Analytics\\Case Study\\Case Study 2\\bellabeat_sleep_clean.csv", index = False)